In [ ]:
import numpy as np

In [ ]:
"""
def log_likelihood(features, target, weights):
    scores = np.dot(features, weights)
    ll = np.sum( target*scores - np.log(1 + np.exp(scores)) )
    return ll
"""

In [ ]:
"""
def logistic_regression(features, target, num_steps, learning_rate, add_intercept = False):
    if add_intercept:
        intercept = np.ones((features.shape[0], 1))
        features = np.hstack((intercept, features))
        
    weights = np.zeros(features.shape[1])
    
    for step in range(num_steps):
        scores = np.dot(features, weights)
        predictions = sigmoid(scores)

        # Update weights with gradient
        output_error_signal = target - predictions
        gradient = np.dot(features.T, output_error_signal)
        weights += learning_rate * gradient
        
        # Print log-likelihood every so often
        if step % 10000 == 0:
            print (log_likelihood(features, target, weights))
        
    return weights
"""

In [ ]:
# sigmoid
"""
def sigmoid(z):

#    Compute the sigmoid of z

#    Arguments:
#    z -- A scalar or numpy array of any size.

#    Return:
#    s -- sigmoid(z)
    
    s = 1 / (1 + np.exp(-z))
    
    return s
"""

In [ ]:
def sigmoid(X, weight):
    z = np.dot(X, weight)
    return 1 / (1 + np.exp(-z))

In [ ]:
def loss(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

In [1]:
def gradient_descent(X, h, y):
    return np.dot(X.T, (h - y)) / y.shape[0]

In [ ]:
def update_weight_loss(weight, learning_rate, gradient):
    return weight - learning_rate * gradient

In [ ]:
def log_likelihood(x, y, weights):
    z = np.dot(x, weights)
    ll = np.sum( y*z - np.log(1 + np.exp(z)) )
    return ll

In [ ]:
def gradient_ascent(X, h, y):
    return np.dot(X.T, y - h)

In [ ]:
def update_weight_mle(weight, learning_rate, gradient):
    return weight + learning_rate * gradient

(4, 1)